In [2]:
from numba import njit
from hftbacktest import GTX, LIMIT, Recorder, HashMapMarketDepthBacktest, BacktestAsset
from hftbacktest.stats import LinearAssetRecord
import numpy as np

beerusdt_20240601 = np.load('hft_db/beerusdt_20240601.npz')['data']

asset = (
    BacktestAsset()
        .data(beerusdt_20240601)
        .linear_asset(1.0)
        .constant_latency(10_000_000, 10_000_000)
        .risk_adverse_queue_model()
        .no_partial_fill_exchange()
        .trading_value_fee_model(0.0002, 0.0007)
        .tick_size(0.00001)
        .lot_size(0.1)
)

In [ ]:
@njit
def orderbookimbalance(hbt, out):
    roi_lb_tick = int(round(30000 / 0.1))
    roi_ub_tick = int(round(90000 / 0.1))

    while hbt.elapse(10 * 1e9) == 0:
        depth = hbt.depth(0)

        mid_price = (depth.best_bid + depth.best_ask) / 2.0

        sum_ask_qty_50bp = 0.0
        sum_ask_qty = 0.0
        for price_tick in range(depth.best_ask_tick, roi_ub_tick + 1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick

            ask_price = price_tick * depth.tick_size
            depth_from_mid = (ask_price - mid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_ask_qty += depth.ask_depth[t]

            if depth_from_mid <= 0.005:
                sum_ask_qty_50bp = sum_ask_qty


        sum_bid_qty_50bp = 0.0
        sum_bid_qty = 0.0
        for price_tick in range(depth.best_bid_tick, roi_lb_tick - 1, -1):
            if price_tick < roi_lb_tick or price_tick > roi_ub_tick:
                continue
            t = price_tick - roi_lb_tick

            bid_price = price_tick * depth.tick_size
            depth_from_mid = (mid_price - bid_price) / mid_price
            if depth_from_mid > 0.01:
                break
            sum_bid_qty += depth.bid_depth[t]

            if depth_from_mid <= 0.005:
                sum_bid_qty_50bp = sum_bid_qty

        imbalance_50bp = sum_bid_qty_50bp - sum_ask_qty_50bp
        imbalance_1pct = sum_bid_qty - sum_ask_qty
        imbalance_tob = depth.bid_depth[depth.best_bid_tick - roi_lb_tick] - depth.ask_depth[depth.best_ask_tick - roi_lb_tick]

        out.append((hbt.current_timestamp, imbalance_tob, imbalance_50bp, imbalance_1pct))
    return True